# Smarter Prompting, Faster: Introducing Opik's Agent Optimizers

Doug Blank, Phd

* Slides are available at: [bit.ly/opik-optimizer-dsblank-slides](https://bit.ly/opik-optimizer-dsblank-slides)
* This notebook is available at: [bit.ly/opik-optimizer-dsblank](https://bit.ly/opik-optimizer-dsblank)

You will need:
1. A Google account, for running a Colab Notebook  - [google.com](https://google.com)
2. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
3. An OpenAI account, for using an LLM
[platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)


## Setup

This pip-install takes about a minute.

In [1]:
%%capture
%pip install opik-optimizer

In [2]:
import opik_optimizer
opik_optimizer.__version__

'0.7.8'

In [3]:
import opik

# Configure Opik
opik.configure()

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "dsblank" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


In [4]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


To save time (and money) durring this demonstration, we have capture the results of a previous run of all of these LLM interactions. Our goal is to make this not cost any money. However, we of course can guarantee that. **Use at your own risk**!

To capture the perviously cached results:

In [5]:
from opik_optimizer.demo.cache import get_litellm_cache

get_litellm_cache("opik-workshop")

Inserted 2865 record(s) in litellm cache


## The Dataset

In these set of experiments, we are going to use the **HotPotQA** dataset. This dataset was designed to be difficult for regular LLMs to handle. This dataset is called a "**multi-hop**" dataset because answering the questions involves multiple reasoning steps and multiple tool calls, where the LLM needs to infer relationships, combine information, or draw conclusions based on the combined context.

Example:

> "What are the capitals of the states that border California?"

You'd need to find which states border California, and then lookup each state's capital.

The dataset has about 113,000 such crowd-sourced questions that are constructed to require the introductory paragraphs of two Wikipedia articles to answer.

[1] The name "HotPot" comes from the restaurant where the authors came up with the idea of the dataset.

In [ ]:
import opik_optimizer

opik_dataset = opik_optimizer.datasets.hotpot_300()

Let's take a look at some dataset items:

In [7]:
rows = opik_dataset.get_items()
rows[0]

{'id': '0196e38c-07c8-79ef-a5aa-f5692df31914',
 'question': 'On what date was the Precision Medicine Initiative announced?',
 'answer': 'January 20, 2015'}

In [8]:
rows[1]

{'id': '0196e38c-07c7-7b6c-92e4-b2fcfe43ffb5',
 'question': 'In addition to Cloud Atlas and the 2016 war drama directed by Vincent Perez, what other film did Stefan Arndt produce?',
 'answer': 'Frantz'}

## Opik Project

All LLM traces in Opik are saved in a "project". We'll put them all in the following project name:

In [9]:
project_name = "optimize-workshop-2025"

## The Metric

Choosing a good metric for optimization is tricky. For these examples, we'll pick one that will allow us to show improvement, and also provide a gradient of scores. In general though, this metric isn't the best for optimization runs.

We'll use "Edit Distance" AKA "Levenshtein Distance":

In [10]:
from opik.evaluation.metrics import LevenshteinRatio
metric = LevenshteinRatio(project_name=project_name)

The metric takes two things: the output of the LLM and the reference (correct answer).

In [11]:
metric.score("Hello", "Hello")

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0196f28d-6043-7a76-becc-1952a662f7a5&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='levenshtein_ratio_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

In [12]:
metric.score("Hello!", "Hello")

ScoreResult(name='levenshtein_ratio_metric', value=0.9090909090909091, reason=None, metadata=None, scoring_failed=False)

The edit distance between "Hello!" and "Hello" is 1. Here is how the .91 is computed:

In [13]:
edit_distance = 1

1 - edit_distance / (len("Hello1") + len("Hello"))


0.9090909090909091

For more information see: [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

## Configuation

To create the necesary configurations for using an Opik Optimizer, you'll need three things:

1. An initial prompt
2. A MetricConfig
3. A TaskConfig

We're going to start with a pretty bad prompt... so we can optimize it!

In [14]:
initial_prompt = "Provide an answer to the question"

The the two configurations:

In [15]:
from opik_optimizer import (
    MetricConfig,
    TaskConfig,
    from_llm_response_text,
    from_dataset_field,
)

metric_config = MetricConfig(
    metric=LevenshteinRatio(project_name=project_name),
    inputs={
        "output": from_llm_response_text(),
        "reference": from_dataset_field(name="answer"),
    },
)

task_config = TaskConfig(
    instruction_prompt=initial_prompt,
    input_dataset_fields=["question"],
    output_dataset_field="answer",
    use_chat_prompt=True,
)

As you can see the MetricConfig is composed of our chosen metric. In addition, we need to know what the inputs will be. The inputs here are actually the outputs from the LLM.

We need two inputs for the metric:
1. The output produced by the LLM (uses a special name)
2. The correct answer (provided by the database item "answer")

The TaskConfig defines how to process a prompt. We need the initial prompt, and the inputs and outputs of the dataset.

In this case, we will use the chat_prompt format as our result.

## FewShotBayesianOptimizer

The FewShotBayesianOptimizer name indicates two things:

1. It will produce Chat Prompts, or FewShot examples as described in the slides.
2. Secondly, it describes how it searches for the best set of these FewShot examples.

To use this optimizer, we import it and create an instance, passing in the project name and model parameters:

In [16]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
)

optimizer = FewShotBayesianOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

### Baseline

Before we optimize this prompt ("Provide an answer to the question") let's see what the bare prompt does by itself on the dataset:

In [17]:
score = optimizer.evaluate_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    prompt=initial_prompt,
    n_samples=100,
)
score

Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ hotpot-300 (100 samples) ─────────────╮
│                                        │
│ Total time:        00:00:18            │
│ Number of samples: 100                 │
│                                        │
│ levenshtein_ratio_metric: 0.1501 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=356108;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196f28d-8e19-7fc5-83f3-9ec458408fd7&dataset_id=0196e38c-066d-7e62-b8f6-ed97eb63387e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

0.15014510084605998

It scored about 16% correct. [I say "percent correct" but because we are using edit distance, that isn't quite accurate. But we can think of it this way.]

Ok, let's optimize that prompt!

In [18]:
result1 = optimizer.optimize_prompt(
    opik_dataset,
    metric_config,
    task_config,
    n_trials=3,
    n_samples=50
)

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

╭─ hotpot-300 (50 samples) ──────────────╮
│                                        │
│ Total time:        00:00:07            │
│ Number of samples: 50                  │
│                                        │
│ levenshtein_ratio_metric: 0.1603 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=475435;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196f28e-c675-7937-8687-f4aa9ee684e3&dataset_id=0196e38c-066d-7e62-b8f6-ed97eb63387e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

...

View the results ]8;id=372528;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196f28e-fd4c-7aa7-b1ae-72ce7fedee19&dataset_id=0196e38c-066d-7e62-b8f6-ed97eb63387e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

╭─ hotpot-300 (50 samples) ──────────────╮
│                                        │
│ Total time:        00:00:05            │
│ Number of samples: 50                  │
│                                        │
│ levenshtein_ratio_metric: 0.4474 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=279946;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196f28f-1447-7045-886e-584c16b35c68&dataset_id=0196e38c-066d-7e62-b8f6-ed97eb63387e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

In [19]:
result1.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         FewShotBayesianOptimizer                                                                     ║
║ Model Used:        openai/gpt-4o-mini (Temp: 0.1)                                                               ║
║ Metric Evaluated:  levenshtein_ratio_metric                                                                     ║
║ Initial Score:     0.1603                                                                                       ║
║ Final Best Score:  0.4474                                                                                       ║
║ Total Improvement: 179.12%                                                                                      ║
║ Rounds Completed:  0                                                                                            ║
║ Stopped Early:     False                                                                                        ║
║                                                                                                                 ║
║                                                                                                                 ║
║ ╭────────────────────────────────── Final Optimized Prompt (Chat Structure) ──────────────────────────────────╮ ║
║ │                                                                                                             │ ║
║ │  Instruction: Provide an answer to the question                                                             │ ║
║ │  ---                                                                                                        │ ║
║ │  System: Provide an answer to the question                                                                  │ ║
║ │                                                                                                             │ ║
║ │  You are an intelligent assistant that learns from few-shot examples provided earlier in the conversation.  │ ║
║ │  Whenever you respond, carefully follow the structure, tone, and format of previous assistant replies,      │ ║
║ │  using them as a guide                                                                                      │ ║
║ │  ---                                                                                                        │ ║
║ │  User:                                                                                                      │ ║
║ │  {                                                                                                          │ ║
║ │    "question": "Which person has a country of origin in Persia, Al-Khazini or Mohamed Hassanein Heikal?"    │ ║
║ │  }                                                                                                          │ ║
║ │                                                                                                             │ ║
║ │  ---                                                                                                        │ ║
║ │  Assistant: Abu al-Fath Khāzini                                                                             │ ║
║ │  ---                                                                                                        │ ║
║ │  User:                                                                                                      │ ║
║ │  {                                                                                                          │ ║
║ │    "question": "What was the score of the National Football League (NFL)'s champion Green Bay Packers       │ ║
║ │  first overall Super Bowl victory since the Orange Bowl?"                                                   │ ║
║ │  }                                                  

What did we find? The result is a series of messages:

In [21]:
result1.details["chat_messages"]

[{'role': 'system',
  'content': 'Provide an answer to the question\n\nYou are an intelligent assistant that learns from few-shot examples provided earlier in the conversation. Whenever you respond, carefully follow the structure, tone, and format of previous assistant replies, using them as a guide'},
 {'role': 'user',
  'content': '\n{\n  "question": "Did Lewis Mumford and Truman Capote share the same nationality?"\n}\n'},
 {'role': 'assistant', 'content': 'yes'},
 {'role': 'user',
  'content': '\n{\n  "question": " The Greenskeeper is a 2002 horror film starring a former Major League Baseball relief pitcher born in which year ?"\n}\n'},
 {'role': 'assistant', 'content': '1974'},
 {'role': 'user',
  'content': '\n{\n  "question": "Which magazine ran longer Women\'s Physique World or Ainslee\'s Magazine?"\n}\n'},
 {'role': 'assistant', 'content': "Ainslee's Magazine"},
 {'role': 'user',
  'content': '\n{\n  "question": "Who is Israeli illusionist, magician, television personality, and

We'll see how we can use those in a few minutes.

## MetaPromptOptimizer

The MetaPromptOptimizer uses a clever idea: have the LLM generate better prompts!

Here is the internal system meta-prompt to have the LLM generate better prompts.

```text
You are an expert prompt engineer. Your task is to improve prompts for any type of task.

Focus on making the prompt more effective by:

1. Being clear and specific about what is expected
2. Providing necessary context and constraints
3. Guiding the model to produce the desired output format
4. Removing ambiguity and unnecessary elements
5. Maintaining conciseness while being complete

Return a JSON array of prompts with the following structure:
{
    "prompts": [
        {
            "prompt": "the improved prompt text",
            "improvement_focus": "what aspect this prompt improves",
            "reasoning": "why this improvement should help"
        }
    ]
}
```

This can work quite well on simpler datasets. It doesn't do so well on HotPot as we will see.

The MetaPromptOptimizer will try a number of rounds to try to find the best prompt.

In [20]:
from opik_optimizer import (
    MetaPromptOptimizer,
)

optimizer = MetaPromptOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",  # Using gpt-4o-mini for evaluation for speed
    max_rounds=1,  # Number of optimization rounds
    num_prompts_per_round=2,  # Number of prompts to generate per round
    improvement_threshold=0.01,  # Minimum improvement required to continue
    temperature=0.1,  # Lower temperature for more focused responses
    max_completion_tokens=5000,  # Maximum tokens for model completion
    num_threads=1,  # Number of threads for parallel evaluation
    subsample_size=20,  # Fixed subsample size
)


We won't do too many rounds, as this is an impossible problem without tools.

In [21]:
result2 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    auto_continue=False,
    n_samples=20,  # Explicitly set
    use_subsample=True,  # Force using subsample for evaluation rounds
)

Evaluation:   0%|          | 0/20 [00:00<?, ?it/s]

╭─ hotpot-300 (20 samples) ──────────────╮
│                                        │
│ Total time:        00:00:01            │
│ Number of samples: 20                  │
│                                        │
│ levenshtein_ratio_metric: 0.2049 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=716751;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196f28f-c6b0-7807-8248-58417e9a3ed7&dataset_id=0196e38c-066d-7e62-b8f6-ed97eb63387e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

Optimizing Prompt:   0%|                    | 0/1 [00:00<?, ?round/s, best_score=0.2049, llm_calls=0] | 0/1 [0…

...

View the results ]8;id=957492;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196f290-6357-7379-a87e-28e857457ebb&dataset_id=0196e38c-066d-7e62-b8f6-ed97eb63387e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

Evaluation:   0%|          | 0/20 [00:00<?, ?it/s]

╭─ hotpot-300 (20 samples) ──────────────╮
│                                        │
│ Total time:        00:00:02            │
│ Number of samples: 20                  │
│                                        │
│ levenshtein_ratio_metric: 0.1984 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=918938;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196f290-6f43-759f-9e75-d8b6dfcdc8cf&dataset_id=0196e38c-066d-7e62-b8f6-ed97eb63387e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

In [22]:
result2.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         MetaPromptOptimizer                                                                          ║
║ Model Used:        openai/gpt-4o-mini (Temp: 0.1)                                                               ║
║ Metric Evaluated:  levenshtein_ratio_metric                                                                     ║
║ Initial Score:     0.2049                                                                                       ║
║ Final Best Score:  0.2049                                                                                       ║
║ Total Improvement: 0.00%                                                                                        ║
║ Rounds Completed:  1                                                                                            ║
║ Stopped Early:     False                                                                                        ║
║                                                                                                                 ║
║                                                                                                                 ║
║ ╭─────────────────────────────────── Final Optimized Prompt (Instruction) ────────────────────────────────────╮ ║
║ │                                                                                                             │ ║
║ │  Provide an answer to the question                                                                          │ ║
║ │                                                                                                             │ ║
║ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ ║
║                                                                                                                 ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

## MiproOptimizer

MIPRO (Multi-Iteration Prompt Optimization) is an optimizer algorithm that refines both prompts and few-shot examples in a multi-stage LLM program. It works by generating, evaluating, and refining prompts to improve language model performance. MIPRO is a more advanced method than simply "prompt hacking," offering real optimization of LLM workflows.

This sophisticated method optimizes both instructions and examples together. Using Bayesian optimization (like the FewShotBayesianOptimizer), it finds the best combinations of both elements. Through multiple testing rounds, it creates an optimized prompt that pairs effective instructions with relevant examples.

For thi first optimization, we aren't going to give it any tools to work with. Let's see how it works:

In [23]:
from opik_optimizer import MiproOptimizer

optimizer = MiproOptimizer(
    model="openai/gpt-4o-mini",  # LiteLLM or OpenAI name
    project_name=project_name,
    temperature=0.1,
    num_threads=16,
)

Remember that we are still starting with the initial prompt:

In [24]:
initial_prompt

'Provide an answer to the question'

In [25]:
result3 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    n_samples=50,
    auto="light",
)


RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 40


==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
These will be used as few-shot example candidates for our program and for creating instructions.

Bootstrapping N=5 sets of demonstrations...
Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 40%|████      | 4/10 [00:04<00:06,  1.08s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/5


 40%|████      | 4/10 [00:03<00:05,  1.04it/s]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 5/5


...


Score: 33.95 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 3'].
Scores so far: [0.18171699527603824, 41.15, 46.44, 39.93, 46.44, 33.95]
Best score so far: 46.44


===== Trial 7 / 7 =====
Average Metric: 17.88 / 40 (44.7%): 100%|██████████| 40/40 [00:04<00:00,  9.74it/s]

2025/05/21 11:19:41 INFO dspy.evaluate.evaluate: Average Metric: 17.881227593903542 / 40 (44.7%)



Score: 44.7 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
Scores so far: [0.18171699527603824, 41.15, 46.44, 39.93, 46.44, 33.95, 44.7]
Best score so far: 46.44


===== Trial 8 / 7 =====
Average Metric: 15.50 / 40 (38.8%): 100%|██████████| 40/40 [00:04<00:00,  8.18it/s]

2025/05/21 11:19:46 INFO dspy.evaluate.evaluate: Average Metric: 15.504488205943364 / 40 (38.8%)



Score: 38.76 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4'].
Scores so far: [0.18171699527603824, 41.15, 46.44, 39.93, 46.44, 33.95, 44.7, 38.76]
Best score so far: 46.44


Returning best identified program with score 46.44!


In [26]:
result3.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         MiproOptimizer                                                                               ║
║ Model Used:        N/A (Temp: N/A)                                                                              ║
║ Metric Evaluated:  levenshtein_ratio_metric                                                                     ║
║ Initial Score:     N/A                                                                                          ║
║ Final Best Score:  46.4400                                                                                      ║
║ Total Improvement: N/A (no initial score)                                                                       ║
║ Rounds Completed:  0                                                                                            ║
║ Stopped Early:     N/A                                                                                          ║
║                                                                                                                 ║
║                                                                                                                 ║
║ ╭─────────────────────────────────── Final Optimized Prompt (Instruction) ────────────────────────────────────╮ ║
║ │                                                                                                             │ ║
║ │  Please provide a concise and informative answer to the following question based on the dataset:            │ ║
║ │                                                                                                             │ ║
║ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ ║
║                                                                                                                 ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [27]:
result3.demonstrations

[{'id': '0196e38c-0741-7f35-aa01-535dd7ea5412',
  'question': 'What is the middle name of the player acquired by the Phoenix Suns from the from the New Jersey Nets during the offseason in 2001-02?',
  'answer': 'Xavier',
  'dspy_uuid': '1d41d06e-edbd-40bc-a6da-1a9374e2a1b4',
  'dspy_split': 'train'},
 {'id': '0196e38c-06a9-7254-aa0c-db3abd0bf3a4',
  'question': 'What caused the plane crash that killed Annette Snell?',
  'answer': 'hail damage and losing thrust on both engines',
  'dspy_uuid': 'a35dc8c5-cff7-4a93-b61e-3b92ec6d4b7a',
  'dspy_split': 'train'},
 {'id': '0196e38c-0727-79de-91af-4cb15f7e00fc',
  'question': 'How are Angostura bitters and Smoking Bishop similar?',
  'answer': 'alcoholic mixture',
  'dspy_uuid': '8c070d54-4da7-474e-9ab9-18c913a13558',
  'dspy_split': 'train'},
 {'id': '0196e38c-06c9-71fe-a883-810bdddd0a2b',
  'question': 'Who wrote the memoir from which the 2014 British biographical romantic drama starring English actor  Vincenzo Leonardo "Enzo" Cilenti was ad

### Agent with Tools

Now we'll try with tools. This will allow multi-prompt optimization.

First, we need a tool. We'll use this one from DSPy:

In [28]:
# Tools:
import dspy

def search_wikipedia(query: str) -> list[str]:
    """
    This agent is used to search wikipedia. It can retrieve additional details
    about a topic.
    """
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(
        query, k=3
    )
    return [x["text"] for x in results]

Let's test it out on a subject:

In [29]:
search_wikipedia("Developmental Robotics")

['Developmental robotics | Developmental robotics (DevRob), sometimes called epigenetic robotics, is a scientific field which aims at studying the developmental mechanisms, architectures and constraints that allow lifelong and open-ended learning of new skills and new knowledge in embodied machines. As in human children, learning is expected to be cumulative and of progressively increasing complexity, and to result from self-exploration of the world in combination with social interaction. The typical methodological approach consists in starting from theories of human and animal development elaborated in fields such as developmental psychology, neuroscience, developmental and evolutionary biology, and linguistics, then to formalize and implement them in robots, sometimes exploring extensions or variants of them. The experimentation of those models in robots allows researchers to confront them with reality, and as a consequence developmental robotics also provides feedback and novel hypo

And it is easy to add the tools to the config. Let's go!

In [30]:
task_config.tools = [search_wikipedia]

result4 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    n_samples=50,
    auto="light",
)


RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 3
valset size: 40


==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
These will be used as few-shot example candidates for our program and for creating instructions.

Bootstrapping N=3 sets of demonstrations...
Bootstrapping set 1/3
Bootstrapping set 2/3
Bootstrapping set 3/3


 40%|████      | 4/10 [00:35<00:53,  8.85s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.

==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.

Proposing instructions...

Proposed Instructions for Predictor 0:

0: Provide an answer to the question

You are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.
Your goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.

To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.
After each tool call, you receive a resulting observation, which gets appended to your trajectory.

When writing next_thought, you may reason about the current situation and plan for future steps.
When

Evaluation:   0%|          | 0/40 [00:00<?, ?it/s]

╭─ hotpot-300 (40 samples) ──────────────╮
│                                        │
│ Total time:        00:00:30            │
│ Number of samples: 40                  │
│                                        │
│ levenshtein_ratio_metric: 0.4786 (avg) │
│                                        │
╰────────────────────────────────────────╯

...


Score: 68.05 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 0', 'Predictor 1: Instruction 2', 'Predictor 1: Few-Shot Set 2'].
Scores so far: [0.4785990375253764, 71.26, 60.9, 69.97, 68.05, 68.05]
Best score so far: 71.26


===== Trial 7 / 7 =====
Average Metric: 28.31 / 40 (70.8%): 100%|██████████| 40/40 [00:33<00:00,  1.19it/s]

2025/05/21 11:25:06 INFO dspy.evaluate.evaluate: Average Metric: 28.311463605257856 / 40 (70.8%)



Score: 70.78 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 1', 'Predictor 1: Instruction 0', 'Predictor 1: Few-Shot Set 1'].
Scores so far: [0.4785990375253764, 71.26, 60.9, 69.97, 68.05, 68.05, 70.78]
Best score so far: 71.26


===== Trial 8 / 7 =====
Average Metric: 27.47 / 40 (68.7%): 100%|██████████| 40/40 [00:20<00:00,  1.91it/s]

2025/05/21 11:25:27 INFO dspy.evaluate.evaluate: Average Metric: 27.467889654492318 / 40 (68.7%)



Score: 68.67 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 0', 'Predictor 1: Instruction 2', 'Predictor 1: Few-Shot Set 2'].
Scores so far: [0.4785990375253764, 71.26, 60.9, 69.97, 68.05, 68.05, 70.78, 68.67]
Best score so far: 71.26


Returning best identified program with score 71.26!


In [31]:
result4.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         MiproOptimizer                                                                               ║
║ Model Used:        N/A (Temp: N/A)                                                                              ║
║ Metric Evaluated:  levenshtein_ratio_metric                                                                     ║
║ Initial Score:     N/A                                                                                          ║
║ Final Best Score:  71.2600                                                                                      ║
║ Total Improvement: N/A (no initial score)                                                                       ║
║ Rounds Completed:  0                                                                                            ║
║ Stopped Early:     N/A                                                                                          ║
║                                                                                                                 ║
║                                                                                                                 ║
║ ╭─────────────────────────────────── Final Optimized Prompt (Instruction) ────────────────────────────────────╮ ║
║ │                                                                                                             │ ║
║ │  As an agent, your task is to provide a detailed answer to the given question based on the trajectory of    │ ║
║ │  your previous reasoning and actions. Begin by analyzing the question and determining what additional       │ ║
║ │  information you need. Use the `search_wikipedia` tool to gather relevant data, and ensure to document      │ ║
║ │  each step of your thought process as you navigate through the task. When you believe you have sufficient   │ ║
║ │  information, use the `finish` tool to conclude the task and prepare your final answer.                     │ ║
║ │                                                                                                             │ ║
║ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ ║
║                                                                                                                 ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [32]:
result4.demonstrations

[{'augmented': True,
  'question': 'Silvano Martina works for a reitred professional footballer who was part of what national team from 2002 to 2011?',
  'trajectory': '',
  'next_thought': 'I need to find out which national football team a retired professional footballer worked for from 2002 to 2011. This likely involves searching for information about Silvano Martina and the footballers he has worked with during that time period.',
  'next_tool_name': 'search_wikipedia',
  'next_tool_args': {'query': 'Silvano Martina retired professional footballer national team 2002 to 2011'}},
 {'augmented': True,
  'question': 'In which island the  RCC Broadcasting Company broadcasts?',
  'trajectory': '[[ ## thought_0 ## ]]\nI need to find out where the RCC Broadcasting Company broadcasts. This information might be available on Wikipedia, so I will search for it there.\n\n[[ ## tool_name_0 ## ]]\nsearch_wikipedia\n\n[[ ## tool_args_0 ## ]]\n{"query": "RCC Broadcasting Company"}\n\n[[ ## observati

## Using Optimized Prompts

Recall:

1. result1 - FewShotBayesianOptimizer
2. result2 - MetaPromptOptimizer
3. result3 - MiproOptimizer (no tools)
4. result4 - MiproOptimizer (with search_wikipedia)

How can we use the optimized results?

For the first one, recall that the fewshot examples are here:

In [33]:
result1.details["chat_messages"]

[{'role': 'system',
  'content': 'Provide an answer to the question\n\nYou are an intelligent assistant that learns from few-shot examples provided earlier in the conversation. Whenever you respond, carefully follow the structure, tone, and format of previous assistant replies, using them as a guide'},
 {'role': 'user',
  'content': '\n{\n  "question": "Which person has a country of origin in Persia, Al-Khazini or Mohamed Hassanein Heikal?"\n}\n'},
 {'role': 'assistant', 'content': 'Abu al-Fath Khāzini'},
 {'role': 'user',
  'content': '\n{\n  "question": "What was the score of the National Football League (NFL)\'s champion Green Bay Packers first overall Super Bowl victory since the Orange Bowl?"\n}\n'},
 {'role': 'assistant', 'content': '35–21'},
 {'role': 'user',
  'content': '\n{\n  "question": " Willis W. Harman worked to foster research through an American non-profit research institute co-founded by what former astronaut?"\n}\n'},
 {'role': 'assistant', 'content': 'Edgar Mitchell

So, once we have those we can do the following:

In [34]:
from litellm.integrations.opik.opik import OpikLogger
import litellm
opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]

def query(question, chat_messages):
    messages = chat_messages[:-1] # Cut off the last one
    # replace it with question in proper format:
    messages.append({'role': 'user', 'content': '{"question": "%s"}"}' % question})

    response = litellm.completion(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=5000,
        messages=messages,
    )
    return response.choices[0].message.content

In [35]:
query("When was David Chalmers born?", result1.details["chat_messages"])

'April 20, 1966'

In [36]:
query("What weighs more: a baby elephant or an SUV?", result1.details["chat_messages"])

'A baby elephant typically weighs more than an SUV.'

If it says "elephant" that is not correct!

Let's try that same question with a tool:

In [37]:
result = result4.details["program"](question="What weighs more: a baby elephant or an SUV?")
result.answer

'An SUV weighs more than a baby elephant.'

Well done optimizer!

We'll now head back to the slides to summarize the workshop.

# Resources

1. [Opik Optimizer Workshop Slides](https://bit.ly/opik-optimizer-dsblank-slides)